## TD($\lambda$) algorithm

## Question 2 and 4 

In [2]:
import os
os.chdir('/Users/Alex/Desktop/Documents_4A/Winter_quarter_1/MS&E_346/RL_book/')

In [8]:
from rl.chapter2.simple_inventory_mrp import SimpleInventoryMRPFinite
user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0
user_gamma = 0.9
si_mrp = SimpleInventoryMRPFinite(
    capacity=user_capacity,
    poisson_lambda=user_poisson_lambda,
    holding_cost=user_holding_cost,
    stockout_cost=user_stockout_cost
)
si_mrp.display_value_function(gamma=user_gamma)

{NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.932,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.511,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.932,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.345,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.345,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.345}


In [16]:
## MY test rep
import rl.iterate as iterate
import rl.td_lambda as td_lambda
import itertools
from pprint import pprint
from rl.chapter10.prediction_utils import fmrp_episodes_stream
from rl.function_approx import learning_rate_schedule
from rl.markov_process import TransitionStep

learning_rate_func: Callable[[int], float] = learning_rate_schedule(
    initial_learning_rate=initial_learning_rate,
    half_life=half_life,
    exponent=exponent
)
    
it: Iterator[ValueFunctionApprox[InventoryState]] = td_lambda_prediction(
    traces=traces,
    approx_0=Tabular(count_to_weight_func=learning_rate_func),
    gamma=user_gamma,
    lambd=0.3
)
    
print("test")
num_transitions = 20000
last_func: ValueFunctionApprox[InventoryState] = last(islice(it, num_transitions))
    
pprint({s: round(last_func.evaluate([s])[0], 3) for s in si_mrp.non_terminal_states})

test
{NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.821,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.333,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.62,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.166,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.223,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.249}


In [12]:
# Implementation with repository
import rl.iterate as iterate
import rl.td_lambda as td_lambda
import itertools
from pprint import pprint
from rl.chapter10.prediction_utils import fmrp_episodes_stream
from rl.function_approx import learning_rate_schedule
from rl.markov_process import TransitionStep


gamma: float = 0.9
episode_length: int = 100
initial_learning_rate: float = 0.03
half_life: float = 1000.0
exponent: float = 0.5
lambda_param = 0.3

episodes: Iterable[Iterable[TransitionStep[S]]] = \
    fmrp_episodes_stream(si_mrp)
    
curtailed_episodes: Iterable[Iterable[TransitionStep[S]]] = \
        (itertools.islice(episode, episode_length) for episode in episodes)

learning_rate_func: Callable[[int], float] = learning_rate_schedule(
    initial_learning_rate=initial_learning_rate,
    half_life=half_life,
    exponent=exponent
)

td_lambda_vfs: Iterator[ValueFunctionApprox[S]] = td_lambda.td_lambda_prediction(
    traces=curtailed_episodes,
    approx_0=Tabular(count_to_weight_func=learning_rate_func),
    gamma=gamma,
    lambd=lambda_param
)
    
num_episodes = 1000

final_td_lambda_vf: ValueFunctionApprox[S] = \
    iterate.last(itertools.islice(td_lambda_vfs, episode_length * num_episodes))
    
pprint({s: round(final_td_lambda_vf(s), 3) for s in si_mrp.non_terminal_states})

{NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.873,
 NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.504,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.818,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.368,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.425,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.555}


In [3]:
from rl.chapter2.simple_inventory_mrp import InventoryState
from rl.function_approx import Tabular
from rl.approximate_dynamic_programming import ValueFunctionApprox
from rl.distribution import Choose
from rl.iterate import last
from rl.monte_carlo import mc_prediction
from itertools import islice
from pprint import pprint
from typing import Iterable, Iterator, TypeVar, Callable

traces = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))

In [4]:
from typing import Iterable, Tuple, Mapping, TypeVar, Iterator, Sequence
from operator import itemgetter
import numpy as np
from itertools import *
from numpy.random import randint
from collections import defaultdict
import itertools

S = TypeVar('S')


In [9]:
import rl.iterate as iterate
import rl.td as td
import itertools
from pprint import pprint
from rl.chapter10.prediction_utils import fmrp_episodes_stream
from rl.chapter10.prediction_utils import unit_experiences_from_episodes
from rl.function_approx import learning_rate_schedule
from rl.markov_process import TransitionStep

user_gamma: float = 0.9
episode_length: int = 100
initial_learning_rate: float = 0.03
half_life: float = 1000.0
exponent: float = 0.5

In [12]:
## Tabular from scratch : no terminal state
episodes: Iterable[Iterable[TransitionStep[S]]] = fmrp_episodes_stream(si_mrp)
    
learning_rate_func: Callable[[int], float] = learning_rate_schedule(
    initial_learning_rate=initial_learning_rate,
    half_life=half_life,
    exponent=exponent
)

def get_td_lamb_value_function(episodes, num_episodes, episode_length, lr, gamma, mp,lamb):
    vc : Mapping[S,float] = {s : 0. for s in mp.non_terminal_states}
    count_lr=0
    count_episode = 0
    for episode in episodes : 
        if count_episode > num_episodes : 
            break
        eligibility_trace = {s : 0. for s in mp.non_terminal_states}
        list_exp = list(itertools.islice(episode,episode_length))
        T0=list_exp[0]
        eligibility_trace[T0.state] = 1
        for T in list_exp :
            for s in mp.non_terminal_states:
                vc[s] = vc[s] + lr(count_lr)*(T.reward + gamma*vc[T.next_state] - vc[T.state])*eligibility_trace[s]
                count_lr += 1
                eligibility_trace[s] = gamma*lamb*eligibility_trace[s] + (T.next_state==s)
        count_episode +=1
    return vc

In [13]:
td_lamb_val = get_td_lamb_value_function(episodes=episodes, num_episodes = 2000, episode_length=200, lr=learning_rate_func, gamma=user_gamma, mp=si_mrp,lamb=0.3)

print("------------- Temporal Difference VALUE FUNCTION --------------")
pprint(td_lamb_val)

------------- Temporal Difference VALUE FUNCTION --------------
{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.53156545109951,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.988479977722708,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.380546342792623,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -29.006590551672453,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.39613749402256,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.436358920574946}


It works great (better than only MC or only TD)

## Question 3 : Relation between MC and TD error

We consider a trace $(S_0,...,S_t,...,S_T)$ with finite horizon $T$, and $V$ is the value function. Thus $S_T$ is a terminal state, and so we have that $V(S_T) = 0$

For a time $0\leq t<T$, the MC error is $G_t - V(S_t)$, with $G_t$ the sum of discounted rewards : $G_t = R_{t+1} + \gamma* G_{t+1}$

Let's prove the result by backward induction :

#### Initialization
At time $t=T-1$, we have that $V(S_{t+1}) = V(S_T) = 0$, and thus $G_{T-1} - V(S_{T-1}) = R_T + \gamma*V(S_T) - V(S_{T-1})$


#### Heredity

Let's consider a time t < T-1 and assume the induction hypothesis true for t+1.

Then $$G_t - V(S_t) = R_{t+1} + \gamma * G_{t+1} - V(S_t) = R_{t+1} + \gamma V(S_{t+1}) - V(S_t) + \gamma * (G_{t+1} - V(S_{t+1}))$$

Using the induction hypothesis, $$G_t - V(S_t) = R_{t+1} + \gamma V(S_{t+1}) - V(S_t) + \sum_{u=t+1}^{T-1}\gamma^{u-(t+1)+1}*(R_{u+1} + \gamma V(S_{u+1}) - V(S_u)) = \sum_{u=t}^{T-1}\gamma^{u-t}*(R_{u+1} + \gamma V(S_{u+1}) - V(S_u))$$

We have proven initialization and heredity, the equality is proven.